(COMPPARALELOGPUSSMC)=

# 5.5 Cómputo en paralelo usando GPUs en un sistema de memoria compartida (SMC)

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

Nota generada a partir de [liga](https://www.dropbox.com/s/yjijtfuky3s5dfz/2.5.Compute_Unified_Device_Architecture.pdf?dl=0)

```{admonition} Al final de esta nota el y la lectora:
:class: tip

*

```

Se presentan códigos y sus ejecuciones en una máquina `<falta colocar>` de la nube de [AWS](https://aws.amazon.com/). Se utilizó la AMI `opt2-aws-educate-openblas-04-04-2021` de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

La máquina `<falta colocar>` tiene las siguientes características:

**Falta ejecutar de acuerdo a la máquina elegida**

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2290.913
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.03
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        8

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.11.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 256GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits
     *-bank:2
          description: DIMM RAM
          physical id: 2
          slot: DIMM 2
          size: 16GiB
          width: 64 bits
     *-bank:3
          description: DIMM RAM
          physical id: 3
          slot: DIMM 3
          size: 16GiB
          width: 64 bits
     *-bank:4
          description: DIMM RAM
          physical id: 4
          slot: DIMM

In [3]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-140 5.4.0-1038-aws #40-Ubuntu SMP Fri Feb 5 23:50:40 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con `import`. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

## *Compute Unified Device Architecture* (CUDA)

### Un poco de historia...

```{margin}

GPGPU es un término que se utilizó para referirse a la programación general en unidades de procesamiento gráfico, hoy en día se conoce simplemente como *GPU programming*. Ver [General-purpose computing on graphics processing units](https://en.wikipedia.org/wiki/General-purpose_computing_on_graphics_processing_units).

```

La industria de videojuegos impulsó el desarrollo de las tarjetas gráficas a una velocidad sin precedente a partir del año 1999 para incrementar el nivel de detalle visual en los juegos de video. Alrededor del 2003 se planteó la posibilidad de utilizar las unidades de procesamiento gráfico para procesamiento en paralelo relacionado con aplicaciones distintas al ambiente de gráficas. A partir del 2006 la empresa [NVIDIA](https://www.nvidia.com/en-us/about-nvidia/) introdujo CUDA, una plataforma GPGPU y un modelo de programación que facilita el procesamiento en paralelo en las GPU's.

Desde entonces las tarjetas gráficas han creado una brecha significativa con las unidades de procesamiento, CPU's. Ver por ejemplo las gráficas que *NVIDIA* publica año tras año y que están relacionadas con el número de operaciones en punto flotante por segundo (FLOPS) y la transferencia de datos en la memoria RAM de la GPU: [from-graphics-processing-to-general-purpose-parallel-computing](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#from-graphics-processing-to-general-purpose-parallel-computing)

```{margin}

La GPU y la CPU están conectadas por una interconexión de nombre [PCI](https://en.wikipedia.org/wiki/Conventional_PCI).

```

Hoy en día se continúa el desarrollo de GPU's con mayor RAM, con mayor capacidad de cómputo y mejor conectividad con la CPU. Estos avances han permitido resolver problemas con mejor exactitud que los resueltos con las CPU's, por ejemplo en el terreno de *deep learning* en reconocimiento de imágenes. Ver [ImageNet Classification with Deep Convolutional Neural Networks](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), [2012: A Breakthrough Year for Deep Learning](https://medium.com/limitlessai/2012-a-breakthrough-year-for-deep-learning-2a31a6796e73).



```{admonition} Observación

Para avances al día de hoy ver [NVIDIA Turing Architecture In-Depth](https://devblogs.nvidia.com/nvidia-turing-architecture-in-depth/), [samsung-amd-rdna-gpu-2021](https://wccftech.com/samsung-amd-rdna-gpu-2021/), [playstation-5-specifications-revealed-but-design-is-still-a-mystery](https://www.theguardian.com/games/2020/mar/19/playstation-5-specifications-revealed-but-design-is-still-a-mystery), [xbox-series-x-tech](https://news.xbox.com/en-us/2020/03/16/xbox-series-x-tech/) y recientemente [IBM Supercomputer Summit Attacks Coronavirus…](https://www.ibm.com/blogs/nordic-msp/ibm-supercomputer-summit-attacks-coronavirus/). 

```

```{margin}

Recuérdese la [taxonomía de Flynn](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy).

```

La arquitectura en la que podemos ubicar a las GPU's es en la de un sistema MIMD y SIMD. De hecho es [SIMT: Simple Instruction Multiple Thread](https://en.wikipedia.org/wiki/Single_instruction,_multiple_threads) en un modelo de sistema de memoria compartida pues "los *threads* en un *warp* cargan la misma instrucción para ser ejecutada".



```{admonition} Definición

Un *warp* en el contexto de GPU *programming* es un conjunto de *threads* y que equivale a $32$, esto es, un *warp* equivale a $32$ *threads*.

```

## ¿Diferencia con la CPU multicore?


<img src="https://dl.dropboxusercontent.com/s/k11qub01w4nvksi/CPU_multicore.png?dl=0" heigth="500" width="500">

**GPU**

<img src="https://dl.dropboxusercontent.com/s/lw9kia12qhwp95r/GPU.png?dl=0" heigth="500" width="500">

```{admonition} Observación
:class: tip

Obsérvese en el dibujo anterior la diferencia en tamaño del caché en la CPU y GPU. También la unidad de control es más pequeña en la GPU.


```

A diferencia de una máquina *multicore* o multi CPU's con la habilidad de lanzar en un instante de tiempo unos cuantos *threads*, por ejemplo cuatro threads en una máquina *quad core*, la GPU puede lanzar cientos o miles de threads en un instante siendo cada core *heavily multithreaded*. Sí hay restricciones en el número de threads que se pueden lanzar en un instante pues las tarjetas gráficas tienen diferentes características (modelo) y arquitecturas (ver [List of NVIDIA GPU's](https://en.wikipedia.org/wiki/List_of_Nvidia_graphics_processing_units)) pero la diferencia es grande. Por ejemplo, el modelo **GT 200** (2009) en un instante puede lanzar 30,720 threads.

Ver [How Graphics Cards Work](https://computer.howstuffworks.com/graphics-card1.htm) y [How Microprocessors Work](https://computer.howstuffworks.com/microprocessor.htm) para más información.

## ¿Otras compañías producen tarjetas gráficas?

Ver por ejemplo la lista de GPU's de [Advanced Micro Devices](https://en.wikipedia.org/wiki/List_of_AMD_graphics_processing_units)

## ¿Si tengo una tarjeta gráfica de AMD puedo correr un programa de CUDA?

No es posible pero entre las alternativas están:

* [OpenCl](https://www.khronos.org/opencl/)

* [OpenACC](https://www.openacc.org/about)

## ¿Si tengo una tarjeta gráfica de NVIDIA un poco antigua puedo correr un programa de CUDA?

Las GPU's producidas por NVIDIA desde 2006 son capaces de correr programas basados en **CUDA C**. La cuestión sería revisar qué *compute capability* tiene tu tarjeta. Ver [Compute Capabilities](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) para las características que tienen las tarjetas más actuales.

## ¿Qué es *CUDA C*?

Es una extensión al lenguaje *C* de programación en el que se utiliza una nueva sintaxis para procesamiento en la GPU. Contiene también una librería *runtime* que define funciones que se ejecutan desde el ***host*** por ejemplo para alojar y desalojar memoria en el ***device***, transferir datos entre la memoria *host* y la memoria *device* o manejar múltiples devices. La librería *runtime* está hecha encima de una API de C de bajo nivel llamada [NVIDIA CUDA Driver API](https://docs.nvidia.com/cuda/cuda-driver-api/index.html) la cual es accesible desde el código. Para información de la API de la librería runtime ver [NVIDIA CUDA Runtime API](https://docs.nvidia.com/cuda/cuda-runtime-api/index.html).

```{admonition} Comentario

La transferencia de datos entre la memoria del *host* a *device* o viceversa es un *bottleneck* fuerte.

```

## ¿A qué se refiere la terminología de host y device?

Host es la máquina multicore o multi CPU's y device es la GPU. Una máquina puede tener múltiples GPU's por lo que tendrá múltiples *devices*.

## Tengo una tarjeta NVIDIA CUDA *capable* ¿qué debo realizar primero?

Realizar instalaciones dependiendo de tu sistema operativo. Ver [Instalación](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA/instalacion) donde además se encontrará información para instalación de [nvidia-docker](https://github.com/NVIDIA/nvidia-docker).

## Instalé lo necesario y al ejecutar en la terminal `nvcc -V` obtengo la versión... ¿cómo puedo probar mi instalación?

1) Obteniendo información del NVIDIA driver ejecutando en la terminal:

```
$nvidia-smi

+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.26       Driver Version: 440.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 750     Off  | 00000000:01:00.0 Off |                  N/A |
| 40%   29C    P8     1W /  38W |     55MiB /   978MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
+-----------------------------------------------------------------------------+
```

2) Compilando y ejecutando el siguiente programa de *CUDA C*:

**Programa de hello world:** `hello_world.cu`

```
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<2,3>>>(); //2 bloques de 3 threads cada uno
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}
```

Compilación: `$nvcc hello_world.cu -o hello_world.out`

Ejecución: `$./hello_world.out`:

```
Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hola del cpu thread
```

3) Haciendo un query a la GPU para ver qué características tiene (lo siguiente es posible ejecutar sólo si se instaló el *CUDA toolkit*): 

/usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery

```
/usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)


Detected 1 CUDA Capable device(s)

Device 0: "Tesla K80"
  CUDA Driver Version / Runtime Version          9.1 / 9.1
  CUDA Capability Major/Minor version number:    3.7
  Total amount of global memory:                 11441 MBytes (11996954624 bytes)
  (13) Multiprocessors, (192) CUDA Cores/MP:     2496 CUDA Cores
  GPU Max Clock rate:                            824 MHz (0.82 GHz)
  Memory Clock rate:                             2505 Mhz
  Memory Bus Width:                              384-bit
  L2 Cache Size:                                 1572864 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(65536), 2D=(65536, 65536), 3D=(4096, 4096, 4096)
  Maximum Layered 1D Texture Size, (num) layers  1D=(16384), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(16384, 16384), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount of shared memory per block:       49152 bytes
  Total number of registers available per block: 65536
  Warp size:                                     32
  Maximum number of threads per multiprocessor:  2048
  Maximum number of threads per block:           1024
  Max dimension size of a thread block (x,y,z): (1024, 1024, 64)
  Max dimension size of a grid size    (x,y,z): (2147483647, 65535, 65535)
  Maximum memory pitch:                          2147483647 bytes
  Texture alignment:                             512 bytes
  Concurrent copy and kernel execution:          Yes with 2 copy engine(s)
  Run time limit on kernels:                     No
  Integrated GPU sharing Host Memory:            No
  Support host page-locked memory mapping:       Yes
  Alignment requirement for Surfaces:            Yes
  Device has ECC support:                        Enabled
  Device supports Unified Addressing (UVA):      Yes
  Supports Cooperative Kernel Launch:            No
  Supports MultiDevice Co-op Kernel Launch:      No
  Device PCI Domain ID / Bus ID / location ID:   0 / 0 / 30
  Compute Mode:
     < Default (multiple host threads can use ::cudaSetDevice() with device simultaneously) >

deviceQuery, CUDA Driver = CUDART, CUDA Driver Version = 9.1, CUDA Runtime Version = 9.1, NumDevs = 1
Result = PASS
```

## ¿Por qué usar CUDA y *CUDA-C* o más general cómputo en la GPU?

* NVIDIA como se mencionó al inicio de la nota fue de las primeras compañías en utilizar la GPU para tareas no relacionadas con el área de gráficos y ha colaborado en el avance del conocimiento de las GPU's y desarrollo de algoritmos y tarjetas gráficas. Otra compañía es [Khronos_Group](https://en.wikipedia.org/wiki/Khronos_Group) por ejemplo, quien actualmente desarrolla [OpenCl](https://www.khronos.org/opencl/).

```{margin}

*Deep learning* se ha utilizado para resolver problemas en *machine learning* típicos. Ejemplos de esto son la clasificación de imágenes, de sonidos o análisis de textos, ver por ejemplo [Practical text analysis using deep learning](https://medium.com/@michael.fire/practical-text-analysis-using-deep-learning-5fb0744efdf9).

```

* El cómputo en la GPU constituye hoy en día (2020) una alternativa fuerte a la implementación de modelos de machine learning ampliamente utilizada por la comunidad científica. En el terreno de cómputo matricial y *deep learning* se encuentran [tensorflow](https://github.com/tensorflow), [caffe](https://github.com/BVLC/caffe), [Theano](https://github.com/Theano/Theano) (aunque ya no es soportado pero está retomado en [pymc-devs/aesara](https://github.com/pymc-devs/aesara)), [Pytorch](https://github.com/pytorch/pytorch) y [keras](https://github.com/keras-team/keras) como ejemplos de lo anterior. 

* Sí hay publicaciones científicas para la implementación de *deep learning* en las CPU's, ver por ejemplo el paper reciente de [SLIDE](https://www.cs.rice.edu/~as143/Papers/SLIDE_MLSys.pdf), [HashingDeepLearning](https://github.com/keroro824/HashingDeepLearning) y las entradas [An algorithm could make CPUs a cheap way to train AI](https://www.engadget.com/2020/03/03/rice-university-slide-cpu-gpu-machine-learning/) y [Deep learning rethink overcomes major obstacle in AI industry](https://www.sciencedaily.com/releases/2020/03/200305135041.htm) . Sin embargo, esta área se encuentra en activa investigación por lo que se han adoptado el uso de implementaciones del *deep learning* utilizando GPU's. 


```{admonition} Comentario

El paper plantea una discusión a realizar con la frase *...change in the state-of-the-art algorithms can render specialized hardware less effective in the future*. Ver por ejemplo [Tensor Cores](https://developer.nvidia.com/tensor-cores), [NVIDIA TENSOR CORES, The Next Generation of Deep Learning](https://www.nvidia.com/en-us/data-center/tensorcore/), [The most powerful computers on the planet: SUMMIT](https://www.ibm.com/thought-leadership/summit-supercomputer/) como ejemplos de hardware especializado en aprendizaje automático con Tensorflow.

*Summit powered by 9,126 IBM Power9 CPUs and over 27,000 NVIDIA V100 Tensor Core GPUS, is able to do 200 quadrillion calculations per second...* [IBM Supercomputer Summit Attacks Coronavirus…](https://www.ibm.com/blogs/nordic-msp/ibm-supercomputer-summit-attacks-coronavirus/).

```

## [CUDA-C](https://docs.nvidia.com/cuda/cuda-c-programming-guide/)

## [CuPy](https://github.com/cupy/cupy)

## [Gputools](https://github.com/nullsatz/gputools) 

Ver [gputools: cran](https://rdrr.io/cran/gputools/)

## Referencias de interés

Para más sobre *Unified Memory* revisar:

* [Even easier introduction to cuda](https://devblogs.nvidia.com/even-easier-introduction-cuda/)

* [Unified memory cuda beginners](https://devblogs.nvidia.com/unified-memory-cuda-beginners/)

Es importante el manejo de errores por ejemplo en el alojamiento de memoria en la GPU. En este caso es útil revisar:

* [How to Query Device Properties and Handle Errors in CUDA C/C++](https://devblogs.nvidia.com/how-query-device-properties-and-handle-errors-cuda-cc/)

En las siguientes preguntas encontramos a personas desarrolladoras de CUDA que las resuelven y resultan muy útiles para continuar con el aprendizaje de CUDA C. Por ejemplo: 

* [Parallel reduction over one axis](https://stackoverflow.com/questions/51526082/cuda-parallel-reduction-over-one-axis)

Otros sistemas de software para el [Heterogeneous computing](https://en.wikipedia.org/wiki/Heterogeneous_computing) son:

* [OpenCl](https://en.wikipedia.org/wiki/OpenCL). Ver [NVIDIA OpenCL SDK Code Samples](https://developer.nvidia.com/opencl) para ejemplos con NVIDIA GPU's.

* [Rth-org/Rth](https://github.com/Rth-org/Rth) y más reciente [matloff/Rth](https://github.com/matloff/Rth). Ver también [rdrr.io matloff/Rth](https://rdrr.io/github/matloff/Rth/f/README.md).

Es posible escribir [kernels](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA_C.ipynb) con CuPy. Ver por ejemplo: [User-Defined Kernels](https://docs-cupy.chainer.org/en/stable/tutorial/kernel.html).

* [CuPy – NumPy-like API accelerated with CUDA](https://docs-cupy.chainer.org/en/stable/index.html)

* [CuPy : NumPy-like API accelerated with CUDA github](https://github.com/cupy/cupy)

Otro paquete para uso de Python+GPU para cómputo matricial es:

* [PyCUDA](https://github.com/inducer/pycuda/) y ver [PyCUDA en el repo de la clase](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/Python/PyCUDA) para más información.

Un paquete para uso de pandas+GPU:

* [Rapids](https://github.com/rapidsai), [cudf](https://github.com/rapidsai/cudf)

Ver [optional-libraries](https://docs-cupy.chainer.org/en/stable/install.html#optional-libraries) para librerías que pueden ser utilizadas con CuPy.


```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Preguntas de comprehensión:**

1)¿Qué factores han determinado un mejor *performance* de una GPU vs una CPU? (contrasta los diseños de una CPU vs una GPU).

2)¿Dentro de qué modelo de arquitectura de máquinas se ubica a la GPU dentro de la taxonomía de Flynn? (tip: tal modelo se le puede comparar con el modelo **Single Program Multiple Data (SPMD)**)

3)¿Qué significan las siglas CUDA y detalla qué es CUDA?.

4)¿Qué es y en qué consiste CUDA C?
    
5)¿Qué es un *kernel*?

6)¿Qué pieza de CUDA se encarga de asignar los bloques de *cuda-threads* a las SM’s?

7)¿Qué características (recursos compartidos, dimensiones, forma de agendar la ejecución en *threads*) tienen los bloques que se asignan a una SM al lanzarse y ejecutarse un *kernel*?

8)¿Qué es un *warp*?

9)Menciona los tipos de memorias que existen en las GPU’s.

10)Supón que tienes una tarjeta GT200 cuyas características son:

* Máximo número de *threads* que soporta una SM en un mismo instante en el tiempo: 1024
* Máximo número de *threads* en un bloque: 512
* Máximo número de bloques por SM: 8
* Número de SM’s que tiene esta GPU: 30

Responde:

a)¿Cuál es la máxima cantidad de *threads* que puede soportar esta GPU en un mismo instante en el tiempo?

b)¿Cuál es la máxima cantidad de *warps* por SM que puede soportar esta GPU en un mismo instante en el tiempo?

c)¿Cuáles configuraciones de bloques y *threads* siguientes aprovechan la máxima cantidad de *warps* en una SM de esta GPU para un mismo instante en el tiempo?
    
1.Una configuración del tipo: bloques de 64 *threads* y 16 bloques.

2.Una configuración del tipo: bloques de 1024 *threads* y 1 bloque.

3.Una configuración del tipo: bloques de 256 *threads* y 4 bloques.

4.Una configuración del tipo: bloques de 512 *threads* y 8 bloques.

\*Debes considerar las restricciones/características de la GPU dadas para responder pues algunas configuraciones infringen las mismas. No estamos considerando *registers* o *shared memory*.


**Referencias:**

1. N. Matloff, Parallel Computing for Data Science. With Examples in R, C++ and CUDA, 2014.

2. D. B. Kirk, W. W. Hwu, Programming Massively Parallel Processors: A Hands-on Approach, Morgan Kaufmann, 2010.

3. NVIDIA,CUDA Programming Guide, NVIDIA Corporation, 2007.

4. B. W. Kernighan, D. M. Ritchie, The C Programming Language, Prentice Hall Software Series, 1988

5. [C/extensiones_a_C/CUDA/](https://github.com/palmoreck/programming-languages/tree/master/C/extensiones_a_C/CUDA)